<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="figures/PDSH-cover-small.png">

*このノートブックには、Jake VanderPlas による [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) からの抜粋が含まれています。コンテンツは利用可能です [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do).*

※テキストは[CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode)で、コードは[CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode)で公開しています。このコンテンツが役立つと思われる場合は、[CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode) による作業のサポートを検討してください!*

<!--ナビゲーション-->
< [Operating on Data in Pandas](03.03-Operations-in-Pandas.ipynb) | [Operating on Data in Pandas](03.03-Operations-in-Pandas.ipynb) | [Operating on Data in Pandas](03.03-Operations-in-Pandas.ipynb) >

<a href="https://colab.research.google.com/github/vitroid/PythonDataScienceHandbook/blob/ja/notebooks/03.04-Missing-Values.ipynb"><img align="left" src="https: //colab.research.google.com/assets/colab-badge.svg" alt="Colab で開く" title="Google Colaboratory で開いて実行する"></a>


# 欠損データの処理

多くのチュートリアルで見られるデータと現実世界のデータの違いは、現実世界のデータがクリーンで均質であることはめったにないということです。
特に、多くの興味深いデータセットには、ある程度のデータが欠落しています。
さらに複雑なことに、データ ソースが異なれば、失われたデータがさまざまな方法で示される場合もあります。

このセクションでは、欠落データに関する一般的な考慮事項について説明し、Pandas がそれをどのように表現するかを説明し、Python で欠落データを処理するための組み込みの Pandas ツールをいくつか紹介します。
ここおよび本全体を通して、一般的に欠損データを *null*、*NaN*、または *NA* 値と呼びます。

## 欠損データ規則におけるトレードオフ

テーブルまたは DataFrame 内の欠落データの存在を示すために開発されたスキームが多数あります。
一般に、それらは 2 つの戦略のいずれかを中心に展開します。欠損値をグローバルに示す *マスク* を使用するか、欠損エントリを示す *センチネル値* を選択します。

マスキング アプローチでは、マスクは完全に別個のブール配列である場合もあれば、値の null ステータスをローカルに示すためにデータ表現の 1 ビットを割り当てる場合もあります。

センチネル アプローチでは、センチネル値は、欠落している整数値を -9999 またはいくつかのまれなビット パターンで示すなど、データ固有の規則にすることも、欠落している浮動小数点値を示すなど、よりグローバルな規則にすることもできます。 IEEE 浮動小数点仕様の一部である特別な値である NaN (Not a Number) を使用します。

これらのアプローチのいずれにもトレードオフがないわけではありません。個別のマスク配列を使用するには、追加のブール配列を割り当てる必要があり、ストレージと計算の両方でオーバーヘッドが追加されます。センチネル値は、表現できる有効な値の範囲を縮小し、CPU および GPU 演算で追加の (多くの場合、最適化されていない) ロジックを必要とする場合があります。 NaN のような一般的な特殊値は、すべてのデータ型で使用できるわけではありません。

普遍的に最適な選択肢が存在しないほとんどの場合と同様に、異なる言語やシステムでは異なる規則が使用されます。
たとえば、R 言語では、欠落データを示すセンチネル値として各データ型内の予約済みビット パターンを使用しますが、SciDB システムでは、NA 状態を示すすべてのセルに付加された追加のバイトを使用します。

## パンダの欠落データ

Pandas が欠損値を処理する方法は、非浮動小数点データ型の NA 値の組み込みの概念を持たない NumPy パッケージへの依存によって制限されます。

Pandas は、個々のデータ型ごとにビット パターンを指定して null であることを示すという点で R のリードに従うことができましたが、このアプローチはかなり扱いにくいことが判明しました。
R には 4 つの基本的なデータ型が含まれていますが、NumPy はこれよりも *はるかに* 多くのデータ型をサポートしています。 .
利用可能なすべての NumPy 型で特定のビット パターンを予約すると、さまざまな型のさまざまな操作を特殊なケースで処理する際に、手に負えない量のオーバーヘッドが発生し、NumPy パッケージの新しいフォークが必要になる可能性さえあります。さらに、小さいデータ型 (8 ビット整数など) の場合、マスクとして使用するビットを犠牲にすると、表現できる値の範囲が大幅に減少します。

NumPy はマスクされた配列をサポートしています。つまり、データを「良い」または「悪い」とマークするために別のブール値マスク配列がアタッチされている配列です。
Pandas はこれから派生する可能性がありますが、ストレージ、計算、およびコード メンテナンスの両方のオーバーヘッドにより、魅力的な選択にはなりません。

これらの制約を念頭に置いて、Pandas は欠損データにセンチネルを使用することを選択し、さらに 2 つの既存の Python null 値を使用することを選択しました: 特別な浮動小数点 ``NaN`` 値と Python ``None`` オブジェクトです。
後述するように、この選択にはいくつかの副作用がありますが、実際には、関心のあるほとんどのケースで適切な妥協点となります。

### ``None``: Pythonic 欠落データ

Pandas が使用する最初のセンチネル値は ``None`` です。これは、Python コードで欠落しているデータによく使用される Python シングルトン オブジェクトです。
これは Python オブジェクトであるため、None は任意の NumPy/Pandas 配列では使用できませんが、データ型が 'object' の配列 (つまり、Python オブジェクトの配列) でのみ使用できます。

In [1]:
import numpy as np
import pandas as pd

In [2]:
vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

この ``dtype=object`` は、配列の内容について NumPy が推測できる最も一般的な型表現は、それらが Python オブジェクトであることを意味します。
この種のオブジェクト配列はいくつかの目的に役立ちますが、データに対する操作はすべて Python レベルで行われ、ネイティブ型の配列で通常見られる高速な操作よりもはるかに多くのオーバーヘッドが発生します。

In [3]:
for dtype in ['object', 'int']:
    print("dtype =", dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

dtype = object
10 loops, best of 3: 78.2 ms per loop

dtype = int
100 loops, best of 3: 3.06 ms per loop



配列で Python オブジェクトを使用するということは、 ``None`` 値を持つ配列全体で ``sum()`` や ``min()`` のような集計を実行すると、通常、エラーが発生することを意味します。

In [4]:
vals1.sum()

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

これは、整数と ``None`` の間の加算が未定義であるという事実を反映しています。

### ``NaN``: 数値データがありません

もう 1 つの欠損データ表現である ``NaN`` (*Not a Number* の頭字語) は異なります。これは、標準の IEEE 浮動小数点表現を使用するすべてのシステムで認識される特別な浮動小数点値です。

In [5]:
vals2 = np.array([1, np.nan, 3, 4]) 
vals2.dtype

dtype('float64')

NumPy がこの配列にネイティブの浮動小数点型を選択したことに注意してください。これは、以前のオブジェクト配列とは異なり、この配列がコンパイル済みコードにプッシュされた高速操作をサポートしていることを意味します。
``NaN`` はデータ ウイルスに少し似ていることに注意してください。接触した他のオブジェクトに感染します。
演算に関係なく、 ``NaN`` を使った算術の結果は別の ``NaN`` になります:

In [6]:
1 + np.nan

nan

In [7]:
0 *  np.nan

nan

これは、値に対する集計が適切に定義されている (つまり、エラーにならない) ことを意味しますが、常に役立つとは限らないことに注意してください。

In [8]:
vals2.sum(), vals2.min(), vals2.max()

(nan, nan, nan)

NumPy は、これらの欠損値を無視するいくつかの特別な集計を提供します。

In [9]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(8.0, 1.0, 4.0)

``NaN`` は具体的には浮動小数点値であることに注意してください。整数、文字列、またはその他の型に相当する NaN 値はありません。

### パンダの NaN と None

``NaN`` と ``None`` の両方にそれぞれの役割があり、Pandas はこの 2 つをほぼ同じように処理するように構築されており、必要に応じて変換します。

In [10]:
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

使用可能なセンチネル値がない型の場合、NA 値が存在する場合、Pandas は自動的に型キャストします。
たとえば、整数配列の値を ``np.nan`` に設定すると、NA に対応するために自動的に浮動小数点型にアップキャストされます。

In [11]:
x = pd.Series(range(2), dtype=int)
x

0    0
1    1
dtype: int64

In [12]:
x[0] = None
x

0    NaN
1    1.0
dtype: float64

整数配列を浮動小数点にキャストするだけでなく、Pandas は ``None`` を ``NaN`` 値に自動的に変換することに注意してください。
(将来、ネイティブ整数 NA を Pandas に追加するという提案があることに注意してください。この記事の執筆時点では、それは含まれていません)。

このタイプの魔法は、R などのドメイン固有言語の NA 値に対するより統一されたアプローチと比べると、少しハックに感じるかもしれませんが、Pandas のセンチネル/キャスティング アプローチは実際には非常にうまく機能し、私の経験では問題を引き起こすことはめったにありません。

次の表に、NA 値が導入されたときの Pandas のアップキャスト規則を示します。

|型クラス | NA保存時の変換 | NA センチネル値 |
|--------------|-----------------------------|------------------------|
| |フローティング |変化なし | ``np.nan`` |
| | ``オブジェクト`` |変化なし | ``None`` または ``np.nan`` |
| | ``整数`` | ``float64`` にキャスト | ``np.nan`` |
| |ブール値 | ``object`` にキャスト | ``None`` または ``np.nan`` |

Pandas では、文字列データは常に ``object`` dtype で保存されることに注意してください。

## Null 値の操作

これまで見てきたように、Pandas は ``None`` と ``NaN`` を欠損値または null 値を示すために本質的に互換性があるものとして扱います。
この規則を容易にするために、Pandas データ構造の null 値を検出、削除、および置換するための便利な方法がいくつかあります。
彼らです：

- ``isnull()``: 欠損値を示すブール値マスクを生成します
- ``notnull()``: ``isnull()`` の反対
- ``dropna()``: データのフィルタリングされたバージョンを返します
- ``fillna()``: 欠損値が埋められた、または補完されたデータのコピーを返します

このセクションの最後に、これらのルーチンの簡単な説明とデモンストレーションを示します。

### null 値の検出
Pandas のデータ構造には、null データを検出するための 2 つの便利なメソッドがあります: ``isnull()`` と ``notnull()`` です。
どちらも、データに対してブール マスクを返します。例えば：

In [13]:
data = pd.Series([1, np.nan, 'hello', None])

In [14]:
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

[Data Indexing and Selection](03.02-Data-Indexing-and-Selection.ipynb) で述べたように、ブール値マスクは ``Series`` または ``DataFrame`` インデックスとして直接使用できます。

In [15]:
data[data.notnull()]

0        1
2    hello
dtype: object

``isnull()`` および ``notnull()`` メソッドは、``DataFrame`` に対して同様のブール値の結果を生成します。

### null 値の削除

以前に使用されたマスキングに加えて、便利なメソッド ``dropna()`` があります。
(NA 値を削除します) と ``fillna()`` (NA 値を埋めます)。 「シリーズ」の場合、
結果は簡単です。

In [16]:
data.dropna()

0        1
2    hello
dtype: object

``DataFrame`` には、さらに多くのオプションがあります。
次の ``DataFrame`` を考えてみましょう:

In [17]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


``DataFrame`` から単一の値をドロップすることはできません。行全体または列全体のみをドロップできます。
アプリケーションに応じて、どちらか一方が必要になる場合があるため、dropna() は ``DataFrame`` に多数のオプションを提供します。

デフォルトでは、 ``dropna()`` は *任意の* null 値が存在するすべての行を削除します:

In [18]:
df.dropna()

,0,1,2
1,2.0,3.0,5


または、別の軸に沿って NA 値をドロップできます。 ``axis=1`` は null 値を含むすべての列を削除します:

In [19]:
df.dropna(axis='columns')

,2
0,2
1,5
2,6


しかし、これはいくつかの良いデータも落とします。 *すべて*の NA 値、または大部分の NA 値を持つ行または列を削除することに興味があるかもしれません。
これは ``how`` または ``thresh`` パラメータで指定できます。これにより、許可する null の数を細かく制御できます。

デフォルトは ``how='any'`` で、(axis`` キーワードに応じて) null 値を含むすべての行または列が削除されます。
*all* null 値の行/列のみをドロップする ``how='all'`` を指定することもできます:

In [20]:
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [21]:
df.dropna(axis='columns', how='all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


より細かく制御するために、thresh パラメータを使用すると、保持する行/列の null 以外の値の最小数を指定できます。

In [22]:
df.dropna(axis='rows', thresh=3)

,0,1,2,3
1,2.0,3.0,5,NaN


ここでは、最初の行と最後の行が削除されています。これは、null 以外の値が 2 つしか含まれていないためです。

### null 値を埋める

NA 値を削除するのではなく、有効な値に置き換えたい場合があります。
この値は、ゼロのような単一の数値である場合もあれば、適切な値からのある種の代入または補間である場合もあります。
``isnull()`` メソッドをマスクとして使用してインプレースでこれを行うことができますが、これは非常に一般的な操作であるため、Pandas は ``fillna()`` メソッドを提供します。 null 値が置き換えられました。

次の ``Series`` を考えてみましょう:

In [23]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

NA エントリをゼロなどの単一の値で埋めることができます。

In [24]:
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

forward-fill を指定して、前の値を前方に伝播できます。

In [25]:
# forward-fill
data.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

または、バックフィルを指定して、次の値を後方に伝播することもできます。

In [26]:
# back-fill
data.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

``DataFrame`` の場合、オプションは似ていますが、塗りつぶしが行われる ``axis`` を指定することもできます:

In [27]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [28]:
df.fillna(method='ffill', axis=1)

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0


フォワード フィル中に前の値が使用できない場合、NA 値が残ることに注意してください。

<!--ナビゲーション-->
< [Operating on Data in Pandas](03.03-Operations-in-Pandas.ipynb) | [Operating on Data in Pandas](03.03-Operations-in-Pandas.ipynb) | [Operating on Data in Pandas](03.03-Operations-in-Pandas.ipynb) >

<a href="https://colab.research.google.com/github/vitroid/PythonDataScienceHandbook/blob/ja/notebooks/03.04-Missing-Values.ipynb"><img align="left" src="https: //colab.research.google.com/assets/colab-badge.svg" alt="Colab で開く" title="Google Colaboratory で開いて実行する"></a>
